# Importing necessary libraries

In [2]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot
import seaborn as sns
import duckdb

# Importing csv file 

*The file will be imported using DuckDB first to serve as the source data before transformed into smaller DataFrames*

In [6]:
from pyspark.sql import SparkSession
# Create a new SparkSession
spark = SparkSession.builder \
    .appName("nyc_taxi_data") \
    .getOrCreate()


In [7]:
nyc_taxi_df = pd.read_csv('./2020_Yellow_Taxi_Trip_Data.csv')

C:\Users\nktun\AppData\Local\Temp\ipykernel_2764\3881252485.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_taxi_df = pd.read_csv('./2020_Yellow_Taxi_Trip_Data.csv')


## Creating a DuckDB table

In [8]:
# Note: duckdb.sql connects to the default in-memory database connection
duckdb.sql("CREATE TABLE nyc_taxi_source_table AS SELECT * FROM nyc_taxi_df")

In [10]:
duckdb.sql("select count(*) from nyc_taxi_df")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     24648499 │
└──────────────┘

Getting column names using Pandas 

In [12]:
nyc_taxi_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

Get an overview of the file

In [13]:
nyc_taxi_df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,2.383893e+07,2.383893e+07,2.464850e+07,2.383893e+07,2.464850e+07,2.464850e+07,2.383893e+07,2.464850e+07,2.464850e+07,2.464850e+07,2.464850e+07,2.464850e+07,2.464850e+07,2.464850e+07,2.464850e+07
mean,1.667327e+00,1.467986e+00,3.527061e+00,1.048557e+00,1.639691e+02,1.611710e+02,1.280403e+00,1.266764e+01,1.071817e+00,5.127972e-01,2.082061e+00,3.038568e-01,2.975557e-01,1.842163e+01,2.212161e+00
std,4.711705e-01,1.112783e+00,3.250359e+02,7.610861e-01,6.675154e+01,7.095615e+01,4.843899e-01,2.740914e+02,1.007181e+02,1.007105e+02,2.610753e+00,1.604868e+00,3.745065e-02,3.402286e+02,8.211437e-01
min,1.000000e+00,0.000000e+00,-3.062000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.259000e+03,-2.700000e+01,-5.000000e-01,-4.932200e+02,-4.000000e+01,-3.000000e-01,-1.260300e+03,-2.500000e+00
25%,1.000000e+00,1.000000e+00,9.900000e-01,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.650000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.920000e+00,0.000000e+00,3.000000e-01,1.430000e+01,2.500000e+00
75%,2.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.340000e+02,2.340000e+02,2.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.760000e+00,0.000000e+00,3.000000e-01,1.980000e+01,2.500000e+00
max,2.000000e+00,9.000000e+00,3.509149e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,9.983100e+05,5.000008e+05,5.000005e+05,1.393560e+03,9.255000e+02,3.000000e-01,1.000004e+06,3.000000e+00


Checking for null values in data

In [14]:
nyc_taxi_df.isnull().sum()

VendorID                 809568
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          809568
trip_distance                 0
RatecodeID               809568
store_and_fwd_flag       809568
PULocationID                  0
DOLocationID                  0
payment_type             809568
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
dtype: int64

Due to the size of the data, DuckDB will be used to query from this 'supposed' database

In [15]:
nyc_taxi_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,01/01/2020 12:28:15 AM,01/01/2020 12:33:03 AM,1.0,1.20,1.0,N,238,239,1.0,6.00,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,01/01/2020 12:35:39 AM,01/01/2020 12:43:04 AM,1.0,1.20,1.0,N,239,238,1.0,7.00,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,01/01/2020 12:47:41 AM,01/01/2020 12:53:52 AM,1.0,0.60,1.0,N,238,238,1.0,6.00,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,01/01/2020 12:55:23 AM,01/01/2020 01:00:14 AM,1.0,0.80,1.0,N,238,151,1.0,5.50,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,01/01/2020 12:01:58 AM,01/01/2020 12:04:16 AM,1.0,0.00,1.0,N,193,193,2.0,3.50,0.5,0.5,0.00,0.0,0.3,4.80,0.0
5,2.0,01/01/2020 12:09:44 AM,01/01/2020 12:10:37 AM,1.0,0.03,1.0,N,7,193,2.0,2.50,0.5,0.5,0.00,0.0,0.3,3.80,0.0
6,2.0,01/01/2020 12:39:25 AM,01/01/2020 12:39:29 AM,1.0,0.00,1.0,N,193,193,1.0,2.50,0.5,0.5,0.01,0.0,0.3,3.81,0.0
7,2.0,12/18/2019 03:27:49 PM,12/18/2019 03:28:59 PM,1.0,0.00,5.0,N,193,193,1.0,0.01,0.0,0.0,0.00,0.0,0.3,2.81,2.5
8,2.0,12/18/2019 03:30:35 PM,12/18/2019 03:31:35 PM,4.0,0.00,1.0,N,193,193,1.0,2.50,0.5,0.5,0.00,0.0,0.3,6.30,2.5
9,1.0,01/01/2020 12:29:01 AM,01/01/2020 12:40:28 AM,2.0,0.70,1.0,N,246,48,1.0,8.00,3.0,0.5,2.35,0.0,0.3,14.15,2.5


In [28]:
duckdb.sql("DESCRIBE nyc_taxi_source_table").show()

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│      column_name      │ column_type │  null   │   key   │ default │ extra │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ VendorID              │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ tpep_pickup_datetime  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ tpep_dropoff_datetime │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ passenger_count       │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ trip_distance         │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ RatecodeID            │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ store_and_fwd_flag    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ PULocationID          │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ DOLocationID          │ BIGINT      │ YES     │ NULL    │ NULL

In [32]:
duckdb.sql("select VendorID, count((vendorID)) from nyc_taxi_source_table group by VendorID")

┌──────────┬─────────────────┐
│ VendorID │ count(vendorID) │
│  double  │      int64      │
├──────────┼─────────────────┤
│      2.0 │        15908366 │
│      1.0 │         7930565 │
│     NULL │               0 │
└──────────┴─────────────────┘

In [38]:
duckdb.sql("select VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, fare_amount from nyc_taxi_source_table order by tpep_pickup_datetime asc limit 30").show()

┌──────────┬────────────────────────┬────────────────────────┬─────────────┐
│ VendorID │  tpep_pickup_datetime  │ tpep_dropoff_datetime  │ fare_amount │
│  double  │        varchar         │        varchar         │   double    │
├──────────┼────────────────────────┼────────────────────────┼─────────────┤
│      2.0 │ 01/01/2003 12:07:17 AM │ 01/01/2003 02:16:59 PM │         0.0 │
│      2.0 │ 01/01/2009 01:04:51 AM │ 01/01/2009 01:20:53 AM │        10.5 │
│      2.0 │ 01/01/2009 01:11:17 AM │ 01/01/2009 01:11:20 AM │         2.5 │
│      2.0 │ 01/01/2009 01:12:10 AM │ 01/01/2009 01:25:57 AM │        10.0 │
│      2.0 │ 01/01/2009 01:17:10 AM │ 01/01/2009 01:52:36 AM │        30.0 │
│      2.0 │ 01/01/2009 01:22:35 AM │ 01/01/2009 01:59:51 AM │        28.0 │
│      2.0 │ 01/01/2009 01:26:40 AM │ 01/01/2009 01:34:39 AM │         8.0 │
│      2.0 │ 01/01/2009 01:28:26 AM │ 01/01/2009 01:36:37 AM │         6.5 │
│      2.0 │ 01/01/2009 01:29:56 AM │ 01/01/2009 01:38:44 AM │         7.0 │

## Parsing date strings into dates so that DuckDB SQL can correctly behave

In [44]:
nyc_taxi_df['tpep_pickup_datetime_formatted'] = pd.to_datetime(nyc_taxi_df['tpep_pickup_datetime'], format="%m/%d/%Y %I:%M:%S %p")

In [ ]:
nyc_taxi_df['tpep_dropoff_datetime_formatted'] = pd.to_datetime(nyc_taxi_df['tpep_dropoff_datetime'], format="%m/%d/%Y %I:%M:%S %p")

In [51]:
# Creating new SQL Source Table that contains the formatted date
duckdb.sql("CREATE TABLE nyc_taxi_source_table AS SELECT * FROM nyc_taxi_df")

*Testing to see if the new column can help filter based on time*

In [52]:
duckdb.sql("select VendorID, tpep_pickup_datetime_formatted, tpep_dropoff_datetime_formatted, fare_amount from nyc_taxi_source_table order by tpep_pickup_datetime_formatted asc limit 30").show()

┌──────────┬────────────────────────────────┬─────────────────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime_formatted │ tpep_dropoff_datetime_formatted │ fare_amount │
│  double  │           timestamp            │            timestamp            │   double    │
├──────────┼────────────────────────────────┼─────────────────────────────────┼─────────────┤
│      2.0 │ 2002-12-31 23:06:55            │ 2002-12-31 23:08:03             │         0.0 │
│      2.0 │ 2003-01-01 00:07:17            │ 2003-01-01 14:16:59             │         0.0 │
│      2.0 │ 2003-03-26 21:07:28            │ 2003-03-27 13:30:46             │         2.5 │
│      2.0 │ 2008-12-31 22:20:23            │ 2008-12-31 22:55:19             │        23.0 │
│      2.0 │ 2008-12-31 23:02:19            │ 2009-01-01 18:14:43             │         5.0 │
│      2.0 │ 2008-12-31 23:02:40            │ 2009-01-01 05:46:33             │        24.0 │
│      2.0 │ 2008-12-31 23:02:48            │ 2009-01-01 16:

*Yep, it worked*

In [55]:
duckdb.sql("select min(tpep_dropoff_datetime_formatted) from nyc_taxi_source_table").show()
duckdb.sql("select max(tpep_dropoff_datetime_formatted) from nyc_taxi_source_table").show()

┌──────────────────────────────────────┐
│ min(tpep_dropoff_datetime_formatted) │
│              timestamp               │
├──────────────────────────────────────┤
│ 2002-12-31 23:08:03                  │
└──────────────────────────────────────┘

┌──────────────────────────────────────┐
│ max(tpep_dropoff_datetime_formatted) │
│              timestamp               │
├──────────────────────────────────────┤
│ 2021-06-10 10:41:42                  │
└──────────────────────────────────────┘



It seems that there are many year values in this dataframe, so it will be wiser to just get a shorter timeframe and conduct analysis 

In [56]:
duckdb.sql("CREATE TABLE nyc_taxi_datamart_2019 AS select * from nyc_taxi_source_table where extract(year from tpep_dropoff_datetime_formatted) = 2019 and extract(year from tpep_pickup_datetime_formatted) = 2019")

In [59]:
nyc_taxi_df_2019 = duckdb.sql("SELECT * FROM nyc_taxi_datamart_2019").df()
nyc_taxi_df_2019.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,tpep_pickup_datetime_formatted,tpep_dropoff_datetime_formatted
0,2.0,12/18/2019 03:27:49 PM,12/18/2019 03:28:59 PM,1.0,0.00,5.0,N,193,193,1.0,0.01,0.0,0.0,0.00,0.0,0.3,2.81,2.5,2019-12-18 15:27:49,2019-12-18 15:28:59
1,2.0,12/18/2019 03:30:35 PM,12/18/2019 03:31:35 PM,4.0,0.00,1.0,N,193,193,1.0,2.50,0.5,0.5,0.00,0.0,0.3,6.30,2.5,2019-12-18 15:30:35,2019-12-18 15:31:35
2,2.0,12/31/2019 11:48:07 PM,12/31/2019 11:53:39 PM,1.0,0.88,1.0,N,41,41,2.0,6.00,0.5,0.5,0.00,0.0,0.3,7.30,0.0,2019-12-31 23:48:07,2019-12-31 23:53:39
3,2.0,12/31/2019 09:39:23 PM,12/31/2019 09:47:34 PM,5.0,2.04,1.0,N,263,41,1.0,8.50,0.5,0.5,2.46,0.0,0.3,14.76,2.5,2019-12-31 21:39:23,2019-12-31 21:47:34
4,2.0,12/31/2019 10:11:41 PM,12/31/2019 10:21:53 PM,5.0,2.95,1.0,N,79,140,1.0,10.50,0.5,0.5,1.00,0.0,0.3,15.30,2.5,2019-12-31 22:11:41,2019-12-31 22:21:53


Removing old datetime columns from the 2019 view, because why keep them there anymore?

In [ ]:
nyc_taxi_df_2019 = nyc_taxi_df_2019.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)


In [64]:
nyc_taxi_df_2019.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,tpep_pickup_datetime_formatted,tpep_dropoff_datetime_formatted
0,2.0,1.0,0.00,5.0,N,193,193,1.0,0.01,0.0,0.0,0.00,0.0,0.3,2.81,2.5,2019-12-18 15:27:49,2019-12-18 15:28:59
1,2.0,4.0,0.00,1.0,N,193,193,1.0,2.50,0.5,0.5,0.00,0.0,0.3,6.30,2.5,2019-12-18 15:30:35,2019-12-18 15:31:35
2,2.0,1.0,0.88,1.0,N,41,41,2.0,6.00,0.5,0.5,0.00,0.0,0.3,7.30,0.0,2019-12-31 23:48:07,2019-12-31 23:53:39
3,2.0,5.0,2.04,1.0,N,263,41,1.0,8.50,0.5,0.5,2.46,0.0,0.3,14.76,2.5,2019-12-31 21:39:23,2019-12-31 21:47:34
4,2.0,5.0,2.95,1.0,N,79,140,1.0,10.50,0.5,0.5,1.00,0.0,0.3,15.30,2.5,2019-12-31 22:11:41,2019-12-31 22:21:53


In [65]:
# Exporting to a new CSV for further analyses
nyc_taxi_df_2019.to_csv('./2019_Yellow_Taxi_Trip_Data')